In [1]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install pip install python-slugify
!{sys.executable} -m pip install pip install bs4
!{sys.executable} -m pip install pip install lxml

  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1277 sha256=25e07933f00cccd226db5310650c03a87f232b490433719187c55ce5f86f8359
  Stored in directory: c:\users\alkhalili\appdata\local\pip\cache\wheels\75\78\21\68b124549c9bdc94f822c02fb9aa3578a669843f9767776bca
Successfully built bs4


In [2]:
import requests, random, logging, urllib.request, json
from bs4 import BeautifulSoup
from tqdm import tqdm
from slugify import slugify

In [3]:
logging.basicConfig(filename='app.log', filemode='w', format='%(name)s - %(levelname)s - %(message)s')

In [4]:
url = 'https://www.investopedia.com/financial-term-dictionary-4769738'

In [7]:
master_links = []

page = urllib.request.urlopen(url).read().decode('utf8','ignore') 
soup = BeautifulSoup(page,"lxml")

for link in soup.find_all('a',{'class': 'terms-bar__link mntl-text-link'},  href = True):

    master_links.append(link.get('href'))

In [8]:
master_links = master_links[0:27]

In [9]:
with open('URL_INDEX_BY_ALPHA.txt', 'w') as f:
    for item in master_links:
        f.write("%s\n" % item)

In [10]:
list_alpha = []

for articleIdx in master_links:
    
    page = urllib.request.urlopen(articleIdx).read().decode('utf8','ignore') 
    soup = BeautifulSoup(page,"lxml")
                         
    for link in soup.find_all('a',{'class': 'dictionary-top300-list__list mntl-text-link'},  href = True):
           
            list_alpha.append(link.get('href'))

In [11]:
with open('FULL_URL_INDEX.txt', 'w') as f:
    for item in list_alpha:
        f.write("%s\n" % item)

In [5]:
logf = open("error.log", "w")

In [13]:
# for article in tqdm(random.sample(list_alpha, 10)):
data = {} #json file
for article in tqdm(list_alpha):
    list_related = []
    body = []
    try:
        
        page = urllib.request.urlopen(article, timeout = 3).read().decode('utf8','ignore')
        soup = BeautifulSoup(page,"lxml")
    
        myTags = soup.find_all('p', {'class': 'comp mntl-sc-block finance-sc-block-html mntl-sc-block-html'})
    
        for link in soup.find_all('a',{'class': 'related-terms__title mntl-text-link'},  href = True):
            list_related.append(link.get('href'))
    
        title = slugify(soup.find('title').get_text(strip=True)) + '.json'
        data['name'] = soup.find('title').get_text(strip=True)
        data['@id'] = article
        data['related'] = list_related
        post = ''
    
        for tag in myTags:
            # body.append(str(tag.get_text(strip=True).encode('utf8', errors='replace'))) #get text content
            body.append(tag.decode_contents()) # get html content

        f = 'data/' + title
        data['body'] = body

        w = open(f, 'w')
        w.write(json.dumps(data))
        w.close()
        
    except:
        
        logf.write("Failed to extract: {0}\n".format(str(article)))
        logging.error("Exception occurred", exc_info=True)
        
    finally:
        
        pass

100%|██████████████████████████████████████████████████████████████████████████████| 6321/6321 [47:22<00:00,  2.22it/s]


   # create RDF from JSON 

In [15]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install pip install rdflib

In [36]:
import os, json, rdflib
from rdflib import URIRef, BNode, Literal, Namespace, Graph
from rdflib.namespace import CSVW, DC, DCAT, DCTERMS, DOAP, FOAF, ODRL2, ORG, OWL, \
                           PROF, PROV, RDF, RDFS, SDO, SH, SKOS, SOSA, SSN, TIME, \
                           VOID, XMLNS, XSD

path_to_json = 'data/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
for link in json_files: 
    with open('data/'+link) as f:
        data = json.load(f)
    #create RDF graph
    g = Graph()
    INVP = Namespace("https://www.investopedia.com/vocab/")
    g.bind("rdfs", RDFS)
    g.bind("schema", SDO)
    g.bind("invp", INVP)
    rdf_content = ''
    termS = URIRef(data['@id'])
    g.add((termS, RDF.type, INVP.Term))
    g.add((termS, SDO.url, termS))
    g.add((termS, RDFS.label, Literal(data['name'])))
    for rel in data['related']:
        g.add((termS, INVP.relates_to, URIRef(rel)))

    separator = ''
    content= separator.join(data['body'])
    g.add((termS, INVP.description, Literal(content)))
    output = '# '+data['name']+ '\n' + g.serialize(format="turtle").decode("utf-8")
    w = open('rdf/'+link.replace('.json','')+'.ttl', 'wb')
    w.write(output.encode("utf8"))
    w.close()